In [1]:
!ls

cloze_task_average_accuracy.xlsx  Compare_data.md	 Tag_cloze_task.ipynb
cloze_task_raw_data.xlsx	  LSTM_probability.xlsx
Compare_data.ipynb		  output.xlsx


In [1]:
import pandas as pd
data = pd.read_excel('LSTM_probability.xlsx')
data_cloze = pd.read_excel('cloze_task_raw_data.xlsx')

In [2]:
data.head()

,item.id,word.serial.no,shown,answer,prob
0,1,1,Введите первое слово,на,0.016503
1,1,2,На,болотах,0.000006
2,1,3,На болотах,оставался,0.000138
3,1,4,На болотах оставался,еще,0.124899
4,1,5,На болотах оставался ещё,лед,0.002193


In [3]:
data_cloze.shape

(64645, 16)

In [4]:
#for each sentence replace 'Введите первое слово'  with 'Start_X' according to sentence id
data_cloze.shown.replace('Введите первое слово', 'Start', inplace=True)
data_cloze[data_cloze.shown=='Start']
d_c_start = data_cloze[['shown', 'item.id']]
d_c_start['item.id'] = d_c_start['item.id'].astype(str)
d_c_start.loc[d_c_start.shown == 'Start', 'XXX'] = d_c_start.shown+'_'+d_c_start['item.id']

/home/semen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/semen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/semen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [5]:
data_cloze['app'] = d_c_start.XXX
import numpy as np
data_cloze['shown'] = np.where(data_cloze['shown']=='Start', data_cloze.app , data_cloze.shown)
data_cloze = data_cloze.drop('app', axis=1)

In [6]:
data_cloze.shape, len(data_cloze.shown.unique())

((64645, 16), 1309)

In [7]:
gb = data_cloze.groupby(['shown', 'answer', 'accuracy']).size()
gb = pd.DataFrame(gb,columns = ['fr'])
gb = gb.reset_index(level=['shown', 'answer', 'accuracy'])
gb.shape

(25528, 4)

In [8]:
gg= data_cloze.groupby(['shown', 'accuracy']).size()
gg = gg.reset_index(level=['shown', 'accuracy']) 
gg.columns = ['shown', 'accuracy', 'all_fr']
part0 = gg[gg.all_fr.max() == gg.all_fr]
max_value = gg[gg.shown == 'В'].all_fr.sum()
print(max_value)
gg.head(), gg.shape

701


(       shown  accuracy  all_fr
 0    Start_1       0.0     138
 1   Start_10       0.0      70
 2   Start_10       1.0       1
 3  Start_100       0.0      16
 4  Start_101       0.0      16, (1978, 3))

In [9]:
gk= gg[['shown', 'all_fr', 'accuracy']].groupby(['shown']).sum()
gk = gk.reset_index(level=['shown'])
gk.columns = ['shown', 'sum_fr', 'accuracy']
gk.head(), gk.shape
gk[gk.accuracy==1.].shape

(679, 3)

In [10]:
gg = gg.merge(gk, left_on='shown', right_on='shown', how='outer')


In [11]:
gg.head()

,shown,accuracy_x,all_fr,sum_fr,accuracy_y
0,Start_1,0.0,138,138,0.0
1,Start_10,0.0,70,71,1.0
2,Start_10,1.0,1,71,1.0
3,Start_100,0.0,16,16,0.0
4,Start_101,0.0,16,16,0.0


In [12]:
print(gg.head())
def normal(x):
    return max_value/x
gg['index_norm'] = gg.sum_fr
gg.index_norm = gg.index_norm.apply(normal)
print(gg.head())
gg['prob'] = np.where(gg.accuracy_x==1., (gg.all_fr*gg.index_norm)/(gg.index_norm*gg.sum_fr), 0)
res = gg[['shown', 'accuracy_x','sum_fr','prob']]
res = res[res.prob!=0.]
print(res.shape)
print(res.head())

       shown  accuracy_x  all_fr  sum_fr  accuracy_y
0    Start_1         0.0     138     138         0.0
1   Start_10         0.0      70      71         1.0
2   Start_10         1.0       1      71         1.0
3  Start_100         0.0      16      16         0.0
4  Start_101         0.0      16      16         0.0
       shown  accuracy_x  all_fr  sum_fr  accuracy_y  index_norm
0    Start_1         0.0     138     138         0.0    5.079710
1   Start_10         0.0      70      71         1.0    9.873239
2   Start_10         1.0       1      71         1.0    9.873239
3  Start_100         0.0      16      16         0.0   43.812500
4  Start_101         0.0      16      16         0.0   43.812500
(679, 4)
        shown  accuracy_x  sum_fr      prob
2    Start_10         1.0      71  0.014085
8   Start_104         1.0      16  0.125000
15  Start_110         1.0     151  0.006623
19  Start_113         1.0     112  0.026786
21  Start_114         1.0      93  0.032258


In [23]:
res[res.prob>=1.]

,shown,accuracy_x,sum_fr,prob
311,В современном обществе семья и школа оказывают...,1.0,17,1.0
421,Во избежание ожогов надо нанести на лицо небол...,1.0,15,1.0
424,Во избежание ожогов надо нанести на лицо небол...,1.0,15,1.0
548,Дрозды и скворцы начали вить семейные гнёзда н...,1.0,15,1.0
738,Зачем ему звонить‚ если откликается спокойный ...,1.0,17,1.0
795,Ирине досталась отдельная комната в двухкомнатной,1.0,16,1.0
815,Какие главные лекарства должны входить,1.0,63,1.0
1184,Олень бродил среди берёз‚ жевал талый снег,1.0,15,1.0
1525,С нескрываемой едкой иронией отзываются они др...,1.0,15,1.0
1552,Собаку‚ виновницу случившегося‚ приказали сечь...,1.0,15,1.0


In [24]:
data.shown.unique().shape


(1170,)

In [25]:
data.shown.replace('Введите первое слово', 'Start', inplace=True)
#data[data.shown=='Start']
d_start = data[['shown', 'item.id']]
d_start['item.id'] = d_start['item.id'].astype(str)
d_start.loc[d_start.shown == 'Start', 'XXX'] = d_start.shown+'_'+d_c_start['item.id']
data['app'] = d_start.XXX
data['shown'] = np.where(data['shown']=='Start', data.app , data.shown)
data = data.drop('app', axis=1)
data.shown.unique().shape


/home/semen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/semen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/semen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

(1185,)

In [26]:
data_m =  data.merge(res, left_on='shown', right_on='shown', how='outer')

In [27]:
data_m.shown.unique().shape

(1206,)

In [28]:
data_m = data_m.dropna()
result = data_m[['shown', 'answer', 'prob_x', 'prob_y']]

In [29]:
check = data_cloze[data_cloze.accuracy==1.][['shown', 'answer']]
check = check.drop_duplicates()
data_result =  result.merge(check, left_on='shown', right_on='shown', how='outer')

In [30]:
data_result = data_result[data_result.answer_x == data_result.answer_y]
data_result = data_result[['shown','answer_x','prob_x','prob_y']]

In [31]:
data_result.columns = ['shown', 'answer', 'prob_by_model', 'prob_by_human']
data_result['R^2'] = (data_result.prob_by_human-data_result.prob_by_model)**2
data

,item.id,word.serial.no,shown,answer,prob
0,1,1,Start_102,на,0.016503
1,1,2,На,болотах,0.000006
2,1,3,На болотах,оставался,0.000138
3,1,4,На болотах оставался,еще,0.124899
4,1,5,На болотах оставался ещё,лед,0.002193
5,1,6,На болотах оставался ещё лёд‚,но,0.039917
6,1,7,На болотах оставался ещё лёд‚ но,на,0.031502
7,1,8,На болотах оставался ещё лёд‚ но на,берегах,0.003701
8,1,9,На болотах оставался ещё лёд‚ но на берегах,реки,0.014212
9,1,10,На болотах оставался ещё лёд‚ но на берегах реки,появилась,0.000670


In [32]:
data_result

,shown,answer,prob_by_model,prob_by_human,R^2
0,На болотах оставался,еще,0.124899,0.029412,9.117721e-03
1,На болотах оставался ещё лёд‚,но,0.039917,0.362963,1.043589e-01
2,На болотах оставался ещё лёд‚ но,на,0.031502,0.043796,1.511303e-04
3,На болотах оставался ещё лёд‚ но на,берегах,0.003701,0.007299,1.294580e-05
4,На болотах оставался ещё лёд‚ но на берегах,реки,0.014212,0.080882,4.444963e-03
5,На болотах оставался ещё лёд‚ но на берегах ре...,трава,0.003874,0.192593,3.561461e-02
11,Он,был,0.048258,0.006711,1.726100e-03
12,Он ловко поддел концом,ножа,0.045508,0.102941,3.298552e-03
13,Он ловко поддел концом ножа замочки‚,и,0.051944,0.355556,9.217985e-02
14,Он ловко поддел концом ножа замочки‚ и,они,0.072151,0.187970,1.341391e-02


In [34]:
measure = data_result[data_result.prob_by_model<data_result.prob_by_human].shape[0]/data_result.shape[0]
print("{}% of data, where prob of cloze task is greater than prob of lstm ".format(round(measure,2)))

0.84% of data, where prob of cloze task is greater than prob of lstm 


In [35]:
data_result['Cross_entropy'] = -(0.0*np.log(data_result.prob_by_model)+1.0*np.log(data_result.prob_by_human))

/home/semen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [36]:
data_result.head()

,shown,answer,prob_by_model,prob_by_human,R^2,Cross_entropy
0,На болотах оставался,еще,0.124899,0.029412,0.009118,3.526361
1,На болотах оставался ещё лёд‚,но,0.039917,0.362963,0.104359,1.013454
2,На болотах оставался ещё лёд‚ но,на,0.031502,0.043796,0.000151,3.128221
3,На болотах оставался ещё лёд‚ но на,берегах,0.003701,0.007299,0.000013,4.919981
4,На болотах оставался ещё лёд‚ но на берегах,реки,0.014212,0.080882,0.004445,2.514760
